# Example for running the default LCA model in AeroMAPS

The default LCA module enables to assess the life-cycle impacts of scenarios with no specific requirements or customizations. It relies on a pre-compiled LCA model stored in a JSON file. This example notebook demonstrates how to set up and run an AeroMAPS simulation using the default LCA model, and visualize the results.

> Note: while convenient, the default LCA model may not cover all specific needs. For example, the default LCA model cannot deal with custom energy carriers or specific processes added by the user. In such cas
> es, users are encouraged to develop custom LCA models tailored to their scenarios and requirements, as described in the [custom LCA example notebook](./life_cycle_assessment_custom.ipynb).

## Import librairies and configuration

In [ ]:
# --- Import libraries ---
%matplotlib widget
import sys
sys.path.insert(0, './utils/')
import matplotlib.pyplot as plt
import time
from plots import plot_stacked_evolution_subplots
plt.style.use("bmh")
from aeromaps import create_process

## Set models and create process

This example notebook covers the ICAO's LTAG integrated scenarios. The corresponding configuration files are provided in the `./data/config_files` directory.

For using the default LCA model, you must declare it in the configuration file with the following lines:
```yaml
life_cycle_assessment:
    lca_model_data_file: "../lca_data/default_model/default_lca_model.json"
    split_by: 'phase'
```

There are two inputs to provide here: 
- `lca_model_data_file`: the path to a `.json` file containing the pre-compiled LCA model
- `split_by` (optional): enables you to split the impacts to get the individual contributions of the air transport components (aircraft production, fuel combustion, etc.). 

> *Note: You may have a look at the [lca_model.json](./data/lca_data/default_model/default_lca_model.json) file to check the different impact categories and LCA parameters for the LCA model provided here.*

In [ ]:
# --- Create process for IS scenario ---
process = create_process(
    configuration_file="data/config_files/config_is2medium_default_lca.yaml",  # <-- select LTAG IS scenario (from 0 to 3)
)

## Run AeroMAPS simulation and assessment

You may now run AeroMAPS and visualize the results.

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

## Visualize LCA results

In [ ]:
# --- Results ---
process_data_vector_outputs = process.data["vector_outputs"]
process_data_float_inputs = process.data["float_inputs"]
process_data_climate = process.data["climate_outputs"]
lca_outputs = process.data["lca_outputs"]
lca_outputs

In [ ]:
# --- Plots ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs)